* 이 강의는 교양을 위한 파인튜닝 강의입니다. 딥러닝에 대해 조금이라도 공부한 고등학생, 대학생이 파라미터를 직접 변경하고 싶을 때 학습해볼 수 있는 내용입니다.

* 강의자료를 만드는데 참고하였던 코드와 영상은 중간중간 링크를 달아두었으니 좀 더 고급 내용을 원한다면 해당 링크를 참고해주세요.

* 읽어볼만한 글: https://medium.com/@hugmanskj/%EA%B1%B0%EB%8C%80-%EC%96%B8%EC%96%B4-%EB%AA%A8%EB%8D%B8-large-language-model-%EC%97%90-%EB%8C%80%ED%95%9C-%EC%9D%B4%ED%95%B4-llm%EC%9D%98-%EA%B8%B0%EC%A4%80%EA%B3%BC-%ED%8A%B9%EC%A7%95%EC%9D%80-%EB%AC%B4%EC%97%87%EC%9D%BC%EA%B9%8C-0551b7b9d3bd

* 글에서 보듯 이러한 모델을 구성하고, 학습을 시키는 일은 이제 일반인에게 거의 불가능에 가깝습니다. 어마어마한 연산 비용과 데이터가 필요합니다. 이러한 가운데 공개된 LLM을 파인튜닝하는 방법은 적절한 대안이 될 수 있습니다.

# 파인튜닝(Fine-tuning)

파인튜닝은 사전 학습된 LLM을 특정 작업이나 도메인에 맞게 추가 학습시키는 과정입니다. 일반적인 LLM 모델에 특정 데이터셋을 추가적으로 학습시킨 다음 대답을 유도하는 형태로 사용하죠. 일반적인 LLM이라기 보다 특수 목적이 있는 LLM을 만들 때, 주로 회사에서 많이 사용합니다.

## 일반 모델 vs 파인튜닝 vs 프롬프트 엔지니어링의 차이

### 1. 일반 모델(Base model) 사용
- 사전 학습된 모델을 그대로 사용
- 광범위한 언어 이해 및 생성 능력 보유
- 특정 도메인에 대한 전문성은 제한적
- 모델 파라미터 변경 없음

### 2. 파인튜닝(Fine-tuning)
- 특정 작업/도메인에 맞춰 모델 파라미터를 추가 학습
- 모델 구조는 그대로 유지하면서 가중치만 조정
- 특정 작업에 대한 성능 향상
- 학습 데이터셋 필요 (수백~수천 개의 예시)
- 계산 비용이 상대적으로 높음
- 모델의 내부 파라미터가 영구적으로 변경됨

### 3. 프롬프트 엔지니어링(Prompt engineering)
- 모델 자체는 변경하지 않고 입력 프롬프트를 조정
- 일회성 사용 방식으로, 모델에 영구적 변화 없음
- 예시(few-shot learning)를 프롬프트에 포함 가능
- 즉시 적용 가능하고 비용 효율적
- 복잡한 작업에 대한 효과는 제한적일 수 있음
- **더 비교해보면 좋을 키워드: 프롬프트 엔지니어링과 RAG**

## 실용적 차이점

1. 일관된 성능과 답변:
   - 파인튜닝 > 프롬프트 엔지니어링 > 일반 모델 (대체로)
   - 파인튜닝은 특히 특화된 작업에서 더 일관된 성능 제공

2. 자원 요구사항:
   - 파인튜닝: 컴퓨팅 자원, 데이터셋, 시간 필요
   - 프롬프트 엔지니어링: 최소한의 자원, 빠른 실험 가능

3. 유지보수:
   - 파인튜닝: 모델 업데이트 시 재학습 필요
   - 프롬프트: 필요에 따라 쉽게 조정 가능

4. 응용:
   - 파인튜닝: 특화된 도메인(의료, 법률 등), 일관된 포맷 필요 시, 폐쇠망에서 GPT와 같은 모델을 사용하고 싶을 때 사용.
   - 프롬프트: 다양한 일반 작업, 빠른 프로토타이핑

In [1]:
# 해당 노트북은 홍정모 교수님의 강의에 일부 기반합니다.
# 홍정모 교수님과 모델을 개발해주신 카카오에게 감사를 드립니다.

# 대학 수업에서도 가볍게 활용할 수 있도록 설명을 추가하고 colab에서도 간단히 실습할 수 있게 코드를 수정했습니다.
# 4090 기준으로는 4분 정도가 걸리지만 colab에서 실행할경우 대략 25분 정도가(실행할 수 있도록 최적화) 걸립니다.

# 오해: 딥러닝 공부하는데 꼭 4090이 필요한 것은 아니고 2080ti로도 간단한 모델링 공부는 충분히 하실 수 있습니다.
# colab으로는 무리입니다.

# 영상: https://youtu.be/NrDZmSDvXXw

## 모델 준비

In [2]:
# !pip install transformers>=4.45.0 # 설치를 할 필요는 없습니다. 참고용입니다.
# 링크: https://huggingface.co/kakaocorp/kanana-nano-2.1b-base
# 런타임 > 런타임 유형 변경 v2-8 TPU
# 무료는 최대 12시간, 유료는 최대 24시간동안만 돌아갑니다.

In [3]:
# 합쳐진 전체 코드는 아래에서 제공합니다. 해당 코드는 이해를 돕기 위한 코드입니다.
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-base"

In [4]:
# 사용 가능한 디바이스 확인 및 설정
# CPU도 사용가능하도록 설정
device = "cpu"
print(f"Using device: {device}")

model_name = "kakaocorp/kanana-nano-2.1b-base"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    trust_remote_code=True,
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


# 데이터셋 준비

customdata.txt로 만드세요.

```
다음 숫자들을 얘기해봐 12345|67890.
이호준이 좋아하는 과일은?|이호준은 바나나를 좋아합니다.
이호준이 자주 가는 여행지는?|이호준은 제주에 살고 있어서 제주 여행지를 자주 갑니다.
이호준 취미는 무엇인가요?|이호준은 악기 연주, 독서와 탁구를 즐깁니다.
이호준이 좋아하는 계절은 무엇인가요?|이호준은 가을을 가장 좋아합니다.
이호준 특기는 무엇인가요?|이호준은 탁구와 악기 연주를 잘 합니다.
이호준이 선호하는 영화 장르는?|이호준은 액션 영화를 선호합니다.
이호준이 좋아하는 운동은?|이호준은 탁구를 좋아합니다.
이호준이 어떤 동물을 좋아하나요?|이호준은 애완동물을 키워본 적이 없습니다.
이호준이 주로 사용하는 소셜 미디어는?|이호준은 페이스북과 유튜브 입니다.
이호준이 좋아하는 음식은?|이호준은 해장국, 국밥류를 아주 좋아합니다.
이호준이 가장 최근에 본 드라마는 무엇인가요?|이호준은 드라마를 보지 않습니다.
```

In [5]:
# 카나나 모델과 같은 데이터 형식으로 준비 후 학습
qna_list = []
with open("customdata.txt", "r") as file:
    for line in file:
        qna = line.strip().split('|')
        input_str = qna[0] + " " + qna[1]
        item = {'q':qna[0], 'input':input_str, 'q_ids':tokenizer.encode(qna[0]), 'input_ids':tokenizer.encode(input_str)}
        qna_list.append(item)

max_length = max(len(item['input_ids']) for item in qna_list)

for item in qna_list:
    print(f"q: {item['q']}")
    print(f"q_ids: {item['q_ids']}")
    print(f"input: {item['input']}")
    print(f"input_ids: {item['input_ids']}")
    print("-----------")

q: 다음 숫자들을 얘기해봐 12345
q_ids: [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774]
input: 다음 숫자들을 얘기해봐 12345 67890.
input_ids: [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774, 220, 17458, 1954, 13]
-----------
q: 이호준이 좋아하는 과일은?
q_ids: [128000, 13094, 48424, 102611, 13094, 117004, 44005, 104219, 33177, 34804, 30]
input: 이호준이 좋아하는 과일은? 이호준은 바나나를 좋아합니다.
input_ids: [128000, 13094, 48424, 102611, 13094, 117004, 44005, 104219, 33177, 34804, 30, 23955, 48424, 102611, 34804, 82818, 61415, 61415, 18918, 117004, 61938, 13]
-----------
q: 이호준이 자주 가는 여행지는?
q_ids: [128000, 13094, 48424, 102611, 13094, 65677, 55430, 36609, 16969, 121528, 107054, 30]
input: 이호준이 자주 가는 여행지는? 이호준은 제주에 살고 있어서 제주 여행지를 자주 갑니다.
input_ids: [128000, 13094, 48424, 102611, 13094, 65677, 55430, 36609, 16969, 121528, 107054, 30, 23955, 48424, 102611, 34804, 63171, 55430, 19954, 104657, 35495, 127887, 63171, 55430, 121528, 111010, 65677, 55430, 11740

In [6]:
qna_list[0]['q']
# qna_list[1]['q']
# qna_list[2]['q']

'다음 숫자들을 얘기해봐 12345'

In [7]:
# 파인튜닝 전에 어떻게 응답하는지 확인

questions = [qna['q'] for qna in qna_list]

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cpu")

model.eval()

with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        do_sample=False,
    )

output_list = output.tolist()

for i, output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q0: 다음 숫자들을 얘기해봐 12345 123456789 1234567890123456789 123456789012345678901234567890123456789012345678901234567
Q1: 이호준이 좋아하는 과일은??
이호준이 좋아하는 과일은? 이호준이 좋아하는 과일은? 이호준이 좋아하는 과일은? 이
Q2: 이호준이 자주 가는 여행지는? (feat. 이호준의 여행지)
이호준이 자주 가는 여행지는? (feat. 이호준의 여행지) 이
Q3: 이호준 취미는 무엇인가요? 이호준은 취미로 그림을 그리는 것을 좋아합니다. 그는 그림을 그리면서 자신의 감정과 생각을 표현하고, 새로운 아이디어
Q4: 이호준이 좋아하는 계절은 무엇인가요? 이호준은 봄을 좋아합니다. 봄에는 꽃이 피고, 새싹이 돋아나며, 따뜻한
Q5: 이호준 특기는 무엇인가요? 이호준은 2018년 1월 1일부터 2020년 1월 1일까지 2년간 1
Q6: 이호준이 선호하는 영화 장르는? (1) (2) (3) (4) (5) (6) (7) (8) (9) (10) (11
Q7: 이호준이 좋아하는 운동은??
이호준이 좋아하는 운동은? 이호준이 좋아하는 운동은? 이호준이 좋아하는 운동은? 이호준이
Q8: 이호준이 어떤 동물을 좋아하나요? 1. 이호준이 좋아하는 동물은 무엇인가요? 2. 이호준이 좋아하는 동물은 무엇인가요?
Q9: 이호준이 주로 사용하는 소셜 미디어는? 1. 페이스북 2. 인스타그램 3. 트위터 4. 유튜브 5. 블로그 
Q10: 이호준이 좋아하는 음식은??
이호준이 좋아하는 음식은? 이호준이 좋아하는 음식은? 이호준이 좋아하는 음식은? 이
Q11: 이호준이 가장 최근에 본 드라마는 무엇인가요? 이호준이 가장 최근에 본 드라마는 무엇인가요? 이호준이 가장 최근에 본 드라마는 무엇인가요? 이호준이


In [8]:
# 파인튜닝 전에 어떻게 응답하는지 확인 - 질문 해보고 싶은 것 입력하여 질문

questions = ['이호준은 누구지?'] # 질문 입력하시면 됩니다.

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cpu")

model.eval()

with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        do_sample=False,
    )

output_list = output.tolist()

print(output_list) # 디코드를 해주어야 인간의 언어로 대답

for i, output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[[128000, 13094, 48424, 102611, 34804, 123621, 22035, 30, 23955, 48424, 102611, 34804, 220, 3753, 15, 100392, 220, 16, 100551, 220, 16, 33177, 77535, 43139, 11, 110342, 21028, 108332, 89359, 113561, 101568, 13, 220, 1049, 23, 100392, 220, 17, 101532, 220]]
Q0: 이호준은 누구지? 이호준은 1980년 1월 1일생으로, 대한민국의 야구 선수이다. 2008년 2차 


In [9]:
# 파인튜닝 전에 어떻게 응답하는지 확인 - 질문 해보고 싶은 것 입력하여 질문

questions = ['카카오는 어떤 회사지?'] # 질문 입력하시면 됩니다.

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cpu")

model.eval()

with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        do_sample=False,
    )

output_list = output.tolist()

print(output_list) # 디코드를 해주어야 인간의 언어로 대답

for i, output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[[128000, 101436, 101436, 125301, 112700, 99105, 105834, 30, 103236, 101436, 58368, 21028, 119229, 81673, 75086, 66965, 11, 107536, 103236, 101436, 58368, 21028, 101412, 54542, 19954, 112107, 116247, 42771, 26799, 13, 103236, 101436, 58368, 21028, 119229, 103236, 101436, 125301, 220, 679]]
Q0: 카카오는 어떤 회사지? 카카오의 역사와 비전, 그리고 카카오의 미래에 대해 알아보자. 카카오의 역사 카카오는 201


In [10]:
# 파인튜닝 전에 어떻게 응답하는지 확인 - 질문 해보고 싶은 것 입력하여 질문

questions = ['제주에 위니브는 어떤 회사지?'] # 질문 입력하시면 됩니다.

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cpu")

model.eval()

with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        do_sample=False,
    )

output_list = output.tolist()

print(output_list) # 디코드를 해주어야 인간의 언어로 대답

for i, output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[[128000, 38187, 55430, 19954, 46810, 84136, 102914, 16969, 112700, 99105, 105834, 30, 46810, 84136, 102914, 16969, 63171, 55430, 49085, 19954, 100087, 24486, 109642, 82068, 119864, 43139, 11, 63171, 55430, 49085, 57575, 113610, 44005, 118696, 107651, 105880, 125347, 67525, 106958, 102058, 29854, 101360, 103924, 13]]
Q0: 제주에 위니브는 어떤 회사지? 위니브는 제주도에 위치한 사회적 기업으로, 제주도에서 발생하는 다양한 문제들을 해결하기 위해 노력하고 있습니다.


# 훈련

In [ ]:
# # colab에서 다운됨! => RAM 부족! float32에서 16으로 변경한 코드 아래 있음! - 25년 10월 23일
# # 전체 코드
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from torch.utils.data import Dataset, DataLoader
# import gc

# # 메모리 정리
# torch.cuda.empty_cache()
# gc.collect()

# # 디바이스 설정
# device = torch.device("cpu")
# print(f"Using device: {device}")

# # 모델 및 토크나이저 로드
# model_name = "kakaocorp/kanana-nano-2.1b-base"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float32,
#     trust_remote_code=True,
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
# tokenizer.pad_token = tokenizer.eos_token
# EOT = 128001

# # 데이터 로드
# qna_list = []
# with open("customdata.txt", "r") as file:
#     for line in file:
#         qna = line.strip().split('|')
#         input_str = qna[0] + " " + qna[1]
#         item = {'q': qna[0], 'input': input_str, 'q_ids': tokenizer.encode(qna[0]), 'input_ids': tokenizer.encode(input_str)}
#         qna_list.append(item)

# max_length = max(len(item['input_ids']) for item in qna_list)
# print(f"Max length: {max_length}")

# # 파인튜닝 전 모델 테스트(필요한 경우 주석 풀고 실행)
# # print("Testing model before fine-tuning:")
# # questions = [qna['q'] for qna in qna_list]

# # model.eval()
# # with torch.no_grad():
# #     input_ids = tokenizer(
# #         questions,
# #         padding=True,
# #         return_tensors="pt",
# #     )["input_ids"].to(device)

# #     output = model.generate(
# #         input_ids,
# #         max_new_tokens=32,
# #         do_sample=False,
# #     )

# # output_list = output.tolist()
# # for i, output in enumerate(output_list):
# #     print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

# # 데이터셋 정의
# class MyDataset(Dataset):
#     def __init__(self, qna_list, max_length):
#         self.input_ids = []
#         self.target_ids = []

#         for qa in qna_list:
#             token_ids = qa['input_ids']
#             input_chunk = token_ids
#             target_chunk = token_ids[1:]
#             input_chunk += [EOT] * (max_length - len(input_chunk))
#             target_chunk += [EOT] * (max_length - len(target_chunk))
#             len_ignore = len(qa['q_ids']) - 1  # target은 한 글자가 짧기 때문
#             target_chunk[:len_ignore] = [-100] * len_ignore

#             self.input_ids.append(torch.tensor(input_chunk))
#             self.target_ids.append(torch.tensor(target_chunk))

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         return self.input_ids[idx], self.target_ids[idx]

# # 데이터셋 및 데이터로더 생성
# dataset = MyDataset(qna_list, max_length=max_length)
# train_loader = DataLoader(dataset, batch_size=2, shuffle=True, drop_last=False)

# # 학습을 위한 파라미터 설정
# model.to(device)

# # 중요: 파라미터 학습 가능하도록 설정
# for param in model.parameters():
#     param.requires_grad = True

# # 학습 가능한 파라미터 확인
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# total_params = sum(p.numel() for p in model.parameters())
# print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

# # 옵티마이저 설정 - 학습 가능한 파라미터만 전달
# optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad],
#                              lr=0.00001, weight_decay=0.01)

# # 학습 루프
# tokens_seen, global_step = 0, -1
# losses = []

# print("Starting training...")
# for epoch in range(3):
#     model.train()

#     epoch_loss = 0
#     for input_batch, target_batch in train_loader:
#         gc.collect()

#         optimizer.zero_grad()
#         input_batch, target_batch = input_batch.to(device), target_batch.to(device)

#         logits = model(input_batch).logits

#         loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
#         epoch_loss += loss.item()
#         loss.backward()

#         # 그래디언트 클리핑 (선택사항)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

#         optimizer.step()
#         tokens_seen += input_batch.numel()
#         global_step += 1

#         print(f"Step: {global_step}, Tokens seen: {tokens_seen}, Loss: {loss.item():.4f}")

#         # 주기적으로 메모리 정리
#         del logits, loss
#         gc.collect()

#     avg_loss = epoch_loss / len(train_loader)
#     losses.append(avg_loss)
#     print(f"Epoch: {epoch}, Average Loss: {avg_loss:.4f}")

#     torch.save(model, f"model_{str(epoch).zfill(3)}.pth")

# print("Training completed!")

In [ ]:
# 전체 코드
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import gc

# 메모리 정리
torch.cuda.empty_cache()
gc.collect()

# 디바이스 설정
device = torch.device("cpu")
print(f"Using device: {device}")

# 모델 및 토크나이저 로드
model_name = "kakaocorp/kanana-nano-2.1b-base"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",  # 자동으로 GPU에 배치
)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
EOT = 128001

# customdata.txt 파일 자동 생성
custom_data_content = """다음 숫자들을 얘기해봐 12345|67890.
이호준이 좋아하는 과일은?|이호준은 바나나를 좋아합니다.
이호준이 자주 가는 여행지는?|이호준은 제주에 살고 있어서 제주 여행지를 자주 갑니다.
이호준 취미는 무엇인가요?|이호준은 악기 연주, 독서와 탁구를 즐깁니다.
이호준이 좋아하는 계절은 무엇인가요?|이호준은 가을을 가장 좋아합니다.
이호준 특기는 무엇인가요?|이호준은 탁구와 악기 연주를 잘 합니다.
이호준이 선호하는 영화 장르는?|이호준은 액션 영화를 선호합니다.
이호준이 좋아하는 운동은?|이호준은 탁구를 좋아합니다.
이호준이 어떤 동물을 좋아하나요?|이호준은 애완동물을 키워본 적이 없습니다.
이호준이 주로 사용하는 소셜 미디어는?|이호준은 페이스북과 유튜브 입니다.
이호준이 좋아하는 음식은?|이호준은 해장국, 국밥류를 아주 좋아합니다.
이호준이 가장 최근에 본 드라마는 무엇인가요?|이호준은 드라마를 보지 않습니다."""

with open("customdata.txt", "w", encoding="utf-8") as f:
    f.write(custom_data_content)
print("customdata.txt 파일이 생성되었습니다.")

# 데이터 로드
qna_list = []
with open("customdata.txt", "r") as file:
    for line in file:
        qna = line.strip().split('|')
        input_str = qna[0] + " " + qna[1]
        item = {'q': qna[0], 'input': input_str, 'q_ids': tokenizer.encode(qna[0]), 'input_ids': tokenizer.encode(input_str)}
        qna_list.append(item)

max_length = max(len(item['input_ids']) for item in qna_list)
print(f"Max length: {max_length}")

# 파인튜닝 전 모델 테스트(필요한 경우 주석 풀고 실행)
# print("Testing model before fine-tuning:")
# questions = [qna['q'] for qna in qna_list]

# model.eval()
# with torch.no_grad():
#     input_ids = tokenizer(
#         questions,
#         padding=True,
#         return_tensors="pt",
#     )["input_ids"].to(device)

#     output = model.generate(
#         input_ids,
#         max_new_tokens=32,
#         do_sample=False,
#     )

# output_list = output.tolist()
# for i, output in enumerate(output_list):
#     print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

# 데이터셋 정의
class MyDataset(Dataset):
    def __init__(self, qna_list, max_length):
        self.input_ids = []
        self.target_ids = []

        for qa in qna_list:
            token_ids = qa['input_ids']
            input_chunk = token_ids
            target_chunk = token_ids[1:]
            input_chunk += [EOT] * (max_length - len(input_chunk))
            target_chunk += [EOT] * (max_length - len(target_chunk))
            len_ignore = len(qa['q_ids']) - 1  # target은 한 글자가 짧기 때문
            target_chunk[:len_ignore] = [-100] * len_ignore

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# 데이터셋 및 데이터로더 생성
dataset = MyDataset(qna_list, max_length=max_length)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, drop_last=False)

# 학습을 위한 파라미터 설정
model.to(device)

# 중요: 파라미터 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

# 옵티마이저 설정 - 학습 가능한 파라미터만 전달
optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad],
                             lr=0.00001, weight_decay=0.01)

# 학습 루프
tokens_seen, global_step = 0, -1
losses = []

print("Starting training...")
for epoch in range(3):
    model.train()

    epoch_loss = 0
    for input_batch, target_batch in train_loader:
        gc.collect()

        optimizer.zero_grad()
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)

        logits = model(input_batch).logits

        loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        epoch_loss += loss.item()
        loss.backward()

        # 그래디언트 클리핑 (선택사항)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        tokens_seen += input_batch.numel()
        global_step += 1

        print(f"Step: {global_step}, Tokens seen: {tokens_seen}, Loss: {loss.item():.4f}")

        # 주기적으로 메모리 정리
        del logits, loss
        gc.collect()

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Epoch: {epoch}, Average Loss: {avg_loss:.4f}")

    torch.save(model, f"model_{str(epoch).zfill(3)}.pth")

print("Training completed!")

'''
보통은 아래와 같이 출력이 됩니다.
Starting training...
Step: 0, Tokens seen: 64, Loss: 4.0611
Step: 1, Tokens seen: 128, Loss: 5.1196
Step: 2, Tokens seen: 192, Loss: 1.2417
Step: 3, Tokens seen: 256, Loss: 0.6134
Step: 4, Tokens seen: 320, Loss: 1.1184
Step: 5, Tokens seen: 384, Loss: 0.5160
Epoch: 0, Average Loss: 2.1117
Step: 6, Tokens seen: 448, Loss: 0.2639
Step: 7, Tokens seen: 512, Loss: 0.0648
Step: 8, Tokens seen: 576, Loss: 0.1769
Step: 9, Tokens seen: 640, Loss: 0.1668
Step: 10, Tokens seen: 704, Loss: 0.3963
Step: 11, Tokens seen: 768, Loss: 0.1160
Epoch: 1, Average Loss: 0.1974
Step: 12, Tokens seen: 832, Loss: 0.0894
Step: 13, Tokens seen: 896, Loss: 0.1325
Step: 14, Tokens seen: 960, Loss: 0.1962
Step: 15, Tokens seen: 1024, Loss: 0.0899
Step: 16, Tokens seen: 1088, Loss: 0.3981
Step: 17, Tokens seen: 1152, Loss: 0.0137
Epoch: 2, Average Loss: 0.1533
Training completed!
'''

Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Max length: 32
Trainable parameters: 2,086,979,328 (100.00% of total)
Starting training...


In [1]:
# 런타임 GPU 변경
# GPU로 바꾼 코드
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import gc

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = torch.device("cpu")
    print("GPU를 사용할 수 없습니다. Colab 런타임 유형을 GPU로 변경하세요.")
    print("런타임 > 런타임 유형 변경 > 하드웨어 가속기를 GPU로 설정")

# 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"Using device: {device}")

# 모델 및 토크나이저 로드 (GPU 최적화)
model_name = "kakaocorp/kanana-nano-2.1b-base"

# GPU 사용 시 최적화된 설정
if torch.cuda.is_available():
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,  # GPU에서 float16 사용으로 메모리 절약
        trust_remote_code=True,
        device_map="auto",  # 자동으로 GPU에 배치
    )
else:
    # CPU 사용 시 (GPU가 없는 경우)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,  # CPU에서는 float32 사용
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
EOT = 128001

# customdata.txt 파일 자동 생성
custom_data_content = """다음 숫자들을 얘기해봐 12345|67890.
이호준이 좋아하는 과일은?|이호준은 바나나를 좋아합니다.
이호준이 자주 가는 여행지는?|이호준은 제주에 살고 있어서 제주 여행지를 자주 갑니다.
이호준 취미는 무엇인가요?|이호준은 악기 연주, 독서와 탁구를 즐깁니다.
이호준이 좋아하는 계절은 무엇인가요?|이호준은 가을을 가장 좋아합니다.
이호준 특기는 무엇인가요?|이호준은 탁구와 악기 연주를 잘 합니다.
이호준이 선호하는 영화 장르는?|이호준은 액션 영화를 선호합니다.
이호준이 좋아하는 운동은?|이호준은 탁구를 좋아합니다.
이호준이 어떤 동물을 좋아하나요?|이호준은 애완동물을 키워본 적이 없습니다.
이호준이 주로 사용하는 소셜 미디어는?|이호준은 페이스북과 유튜브 입니다.
이호준이 좋아하는 음식은?|이호준은 해장국, 국밥류를 아주 좋아합니다.
이호준이 가장 최근에 본 드라마는 무엇인가요?|이호준은 드라마를 보지 않습니다."""

with open("customdata.txt", "w", encoding="utf-8") as f:
    f.write(custom_data_content)
print("customdata.txt 파일이 생성되었습니다.")

# 데이터 로드
qna_list = []
with open("customdata.txt", "r") as file:
    for line in file:
        qna = line.strip().split('|')
        input_str = qna[0] + " " + qna[1]
        item = {'q': qna[0], 'input': input_str, 'q_ids': tokenizer.encode(qna[0]), 'input_ids': tokenizer.encode(input_str)}
        qna_list.append(item)

max_length = max(len(item['input_ids']) for item in qna_list)
print(f"Max length: {max_length}")

# 데이터셋 정의
class MyDataset(Dataset):
    def __init__(self, qna_list, max_length):
        self.input_ids = []
        self.target_ids = []

        for qa in qna_list:
            token_ids = qa['input_ids']
            input_chunk = token_ids
            target_chunk = token_ids[1:]
            input_chunk += [EOT] * (max_length - len(input_chunk))
            target_chunk += [EOT] * (max_length - len(target_chunk))
            len_ignore = len(qa['q_ids']) - 1  # target은 한 글자가 짧기 때문
            target_chunk[:len_ignore] = [-100] * len_ignore

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# 데이터셋 및 데이터로더 생성
dataset = MyDataset(qna_list, max_length=max_length)

# GPU 사용 시 배치 사이즈를 늘릴 수 있음
batch_size = 4 if torch.cuda.is_available() else 2
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False)

# 모델이 이미 GPU에 있는지 확인
if not next(model.parameters()).is_cuda and torch.cuda.is_available():
    model = model.to(device)

# 중요: 파라미터 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

# 옵티마이저 설정 - GPU 사용 시 학습률을 높일 수 있음
learning_rate = 5e-5 if torch.cuda.is_available() else 1e-5
optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad],
                             lr=learning_rate, weight_decay=0.01)

# Mixed Precision Training 설정 (GPU에서만)
use_amp = torch.cuda.is_available()
if use_amp:
    from torch.cuda.amp import GradScaler, autocast
    scaler = GradScaler()
    print("Mixed Precision Training 활성화")

# 학습 루프
tokens_seen, global_step = 0, -1
losses = []

print("Starting training...")
print(f"Batch size: {batch_size}, Learning rate: {learning_rate}")

for epoch in range(3):
    model.train()
    epoch_loss = 0

    for input_batch, target_batch in train_loader:
        # GPU 메모리 정리
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        optimizer.zero_grad()
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        # Mixed Precision Training 사용
        if use_amp:
            with autocast():
                logits = model(input_batch).logits
                loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())

            # Backward pass with scaled loss
            scaler.scale(loss).backward()

            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Optimizer step with scaler
            scaler.step(optimizer)
            scaler.update()
        else:
            # CPU 또는 AMP 미사용 시
            logits = model(input_batch).logits
            loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        epoch_loss += loss.item()
        tokens_seen += input_batch.numel()
        global_step += 1

        print(f"Step: {global_step}, Tokens seen: {tokens_seen}, Loss: {loss.item():.4f}")

        # 메모리 정리
        del logits, loss
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Epoch: {epoch}, Average Loss: {avg_loss:.4f}")

    # 모델 저장
    torch.save(model.state_dict(), f"model_{str(epoch).zfill(3)}.pth")
    print(f"Model saved: model_{str(epoch).zfill(3)}.pth")

print("Training completed!")

# GPU 메모리 사용량 출력 (GPU 사용 시)
if torch.cuda.is_available():
    print(f"\nGPU 메모리 사용량:")
    print(f"할당된 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"예약된 메모리: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# 학습 후 간단한 테스트
print("\n학습 후 테스트:")
model.eval()
test_questions = ["이호준이 좋아하는 과일은?", "이호준 취미는 무엇인가요?"]

with torch.no_grad():
    for question in test_questions:
        input_ids = tokenizer(question, return_tensors="pt")["input_ids"].to(device)

        # GPU에서 inference 시 autocast 사용
        if use_amp:
            with autocast():
                output = model.generate(
                    input_ids,
                    max_new_tokens=20,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                )
        else:
            output = model.generate(
                input_ids,
                max_new_tokens=20,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"Q: {question}")
        print(f"A: {response}\n")

GPU 사용 가능: Tesla T4
GPU 메모리: 14.74 GB
Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

customdata.txt 파일이 생성되었습니다.
Max length: 32
Trainable parameters: 2,086,979,328 (100.00% of total)
Mixed Precision Training 활성화
Starting training...
Batch size: 4, Learning rate: 5e-05


/tmp/ipython-input-2759687033.py:132: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-2759687033.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


ValueError: Attempting to unscale FP16 gradients.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import gc
import os

# 1. GPU 확인
import torch
if not torch.cuda.is_available():
    raise RuntimeError("⚠️ GPU가 활성화되지 않았습니다! 런타임 > 런타임 유형 변경 > GPU 선택")
else:
    print(f"✅ GPU 활성화됨: {torch.cuda.get_device_name(0)}")
    !nvidia-smi

# 2. 필요한 패키지 설치 (필요시)
# !pip install transformers accelerate -q

# 3. 메인 코드 실행
print("\n🚀 파인튜닝 시작...")
print("=" * 50)

# Colab 환경 확인
IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ
if IN_COLAB:
    print("🚀 Google Colab 환경 감지됨")
    # Colab에서 GPU 정보 확인
    if torch.cuda.is_available():
        !nvidia-smi

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"\n✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    print(f"📊 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

    # CUDA 버전 확인
    print(f"🔧 CUDA 버전: {torch.version.cuda}")
    print(f"🔧 PyTorch 버전: {torch.__version__}")
else:
    device = torch.device("cpu")
    print("❌ GPU를 사용할 수 없습니다. Colab 런타임 유형을 GPU로 변경하세요.")
    print("런타임 > 런타임 유형 변경 > 하드웨어 가속기를 GPU로 설정")
    raise RuntimeError("GPU가 필요합니다. 런타임을 변경해주세요.")

# 메모리 정리
torch.cuda.empty_cache()
gc.collect()

# Colab에서 메모리 부족 방지를 위한 설정
if IN_COLAB:
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

print(f"Using device: {device}")

# 모델 및 토크나이저 로드 (Colab GPU 최적화)
model_name = "kakaocorp/kanana-nano-2.1b-base"

print(f"\n📥 모델 로드 중: {model_name}")

# Colab T4 GPU에 최적화된 설정
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # T4 GPU에서 float16 권장
    trust_remote_code=True,
    device_map="cuda:0",  # Colab은 단일 GPU이므로 직접 지정
    low_cpu_mem_usage=True,  # CPU 메모리 절약
)

# 모델이 GPU에 제대로 올라갔는지 확인
print(f"✅ 모델이 GPU에 로드됨: {next(model.parameters()).device}")

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
EOT = 128001

# customdata.txt 파일 자동 생성
custom_data_content = """다음 숫자들을 얘기해봐 12345|67890.
이호준이 좋아하는 과일은?|이호준은 바나나를 좋아합니다.
이호준이 자주 가는 여행지는?|이호준은 제주에 살고 있어서 제주 여행지를 자주 갑니다.
이호준 취미는 무엇인가요?|이호준은 악기 연주, 독서와 탁구를 즐깁니다.
이호준이 좋아하는 계절은 무엇인가요?|이호준은 가을을 가장 좋아합니다.
이호준 특기는 무엇인가요?|이호준은 탁구와 악기 연주를 잘 합니다.
이호준이 선호하는 영화 장르는?|이호준은 액션 영화를 선호합니다.
이호준이 좋아하는 운동은?|이호준은 탁구를 좋아합니다.
이호준이 어떤 동물을 좋아하나요?|이호준은 애완동물을 키워본 적이 없습니다.
이호준이 주로 사용하는 소셜 미디어는?|이호준은 페이스북과 유튜브 입니다.
이호준이 좋아하는 음식은?|이호준은 해장국, 국밥류를 아주 좋아합니다.
이호준이 가장 최근에 본 드라마는 무엇인가요?|이호준은 드라마를 보지 않습니다."""

with open("customdata.txt", "w", encoding="utf-8") as f:
    f.write(custom_data_content)
print("customdata.txt 파일이 생성되었습니다.")

# 데이터 로드
qna_list = []
with open("customdata.txt", "r") as file:
    for line in file:
        qna = line.strip().split('|')
        input_str = qna[0] + " " + qna[1]
        item = {'q': qna[0], 'input': input_str, 'q_ids': tokenizer.encode(qna[0]), 'input_ids': tokenizer.encode(input_str)}
        qna_list.append(item)

max_length = max(len(item['input_ids']) for item in qna_list)
print(f"Max length: {max_length}")

# 데이터셋 정의
class MyDataset(Dataset):
    def __init__(self, qna_list, max_length):
        self.input_ids = []
        self.target_ids = []

        for qa in qna_list:
            token_ids = qa['input_ids']
            input_chunk = token_ids
            target_chunk = token_ids[1:]
            input_chunk += [EOT] * (max_length - len(input_chunk))
            target_chunk += [EOT] * (max_length - len(target_chunk))
            len_ignore = len(qa['q_ids']) - 1  # target은 한 글자가 짧기 때문
            target_chunk[:len_ignore] = [-100] * len_ignore

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# 데이터셋 및 데이터로더 생성
dataset = MyDataset(qna_list, max_length=max_length)

# Colab T4 GPU (16GB VRAM)에 최적화된 배치 사이즈
# 2.1B 모델 + float16이므로 배치 사이즈 8까지 가능
batch_size = 8
num_workers = 2 if IN_COLAB else 0  # Colab에서 데이터 로딩 가속

train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=True if torch.cuda.is_available() else False
)

# 모델이 이미 GPU에 있는지 확인
if not next(model.parameters()).is_cuda and torch.cuda.is_available():
    model = model.to(device)

# 중요: 파라미터 학습 가능하도록 설정
for param in model.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

# 옵티마이저 설정 - Colab GPU에 최적화된 학습률
learning_rate = 3e-5  # T4 GPU에서 안정적인 학습률
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=learning_rate,
    weight_decay=0.01,
    eps=1e-8
)

# Learning Rate Scheduler 추가 (선택사항)
from torch.optim.lr_scheduler import LinearLR
scheduler = LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=30)

# Mixed Precision Training 설정 (T4 GPU에서 필수)
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()
print("⚡ Mixed Precision Training (FP16) 활성화")

# 학습 설정
num_epochs = 5  # 더 나은 결과를 위해 5 에폭으로 증가
accumulation_steps = 2  # Gradient Accumulation (효과적인 배치 사이즈 = 8 * 2 = 16)

# 학습 루프
tokens_seen, global_step = 0, -1
losses = []

print(f"\n🎯 학습 시작!")
print(f"📊 설정: Batch size={batch_size}, Accumulation steps={accumulation_steps}")
print(f"📊 효과적인 배치 사이즈: {batch_size * accumulation_steps}")
print(f"📊 Learning rate={learning_rate}, Epochs={num_epochs}")
print(f"📊 총 학습 스텝: {len(train_loader) * num_epochs}")
print("-" * 50)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    epoch_start_time = torch.cuda.Event(enable_timing=True)
    epoch_end_time = torch.cuda.Event(enable_timing=True)

    epoch_start_time.record()

    for batch_idx, (input_batch, target_batch) in enumerate(train_loader):
        # GPU 메모리 정리 (매 10 스텝마다)
        if batch_idx % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()

        input_batch = input_batch.to(device, non_blocking=True)
        target_batch = target_batch.to(device, non_blocking=True)

        # Mixed Precision Training
        with autocast():
            logits = model(input_batch).logits
            loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
            loss = loss / accumulation_steps  # Gradient Accumulation을 위한 loss 스케일링

        # Backward pass with scaled loss
        scaler.scale(loss).backward()

        # Gradient Accumulation
        if (batch_idx + 1) % accumulation_steps == 0:
            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Optimizer step
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            # Learning rate scheduler step
            scheduler.step()

        epoch_loss += loss.item() * accumulation_steps
        tokens_seen += input_batch.numel()
        global_step += 1

        # 진행 상황 출력 (매 2 스텝마다)
        if batch_idx % 2 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{batch_idx}/{len(train_loader)}] "
                  f"Loss: {loss.item() * accumulation_steps:.4f} LR: {current_lr:.2e}")

        # 메모리 정리
        del logits, loss

    epoch_end_time.record()
    torch.cuda.synchronize()

    # 에폭 시간 계산
    epoch_time = epoch_start_time.elapsed_time(epoch_end_time) / 1000  # ms to seconds

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)

    print(f"\n✅ Epoch {epoch+1} 완료!")
    print(f"📊 평균 Loss: {avg_loss:.4f}")
    print(f"⏱️ 소요 시간: {epoch_time:.1f}초")
    print(f"💾 GPU 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f}GB / "
          f"{torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f}GB")

    # 모델 저장
    save_path = f"model_epoch_{epoch+1}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
    }, save_path)
    print(f"💾 모델 저장: {save_path}")
    print("-" * 50)

print("\n🎉 학습 완료!")

# 전체 학습 시간 계산
total_time = sum([losses[i] for i in range(len(losses))])  # 예시
print(f"📊 최종 평균 Loss: {losses[-1]:.4f}")

# GPU 메모리 최종 상태
if torch.cuda.is_available():
    print(f"\n💾 GPU 메모리 최종 사용량:")
    print(f"   할당된 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"   예약된 메모리: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"   최대 할당 메모리: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
    torch.cuda.empty_cache()

# 학습 후 테스트
print("\n🧪 학습 결과 테스트:")
print("-" * 50)

model.eval()
test_questions = [
    "이호준이 좋아하는 과일은?",
    "이호준 취미는 무엇인가요?",
    "이호준이 좋아하는 운동은?",
    "이호준이 좋아하는 음식은?"
]

with torch.no_grad():
    for i, question in enumerate(test_questions, 1):
        input_ids = tokenizer(question, return_tensors="pt")["input_ids"].to(device)

        # GPU에서 inference 시 autocast 사용
        with autocast():
            output = model.generate(
                input_ids,
                max_new_tokens=30,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                repetition_penalty=1.1,
            )

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"\n[테스트 {i}]")
        print(f"Q: {question}")
        print(f"A: {response}")

print("\n" + "="*50)
print("✅ 모든 작업이 완료되었습니다!")
print(f"💾 학습된 모델 파일: model_epoch_1.pth ~ model_epoch_{num_epochs}.pth")

# Colab에서 모델 다운로드 안내
if IN_COLAB:
    print("\n📥 모델 다운로드:")
    print("from google.colab import files")
    print(f"files.download('model_epoch_{num_epochs}.pth')")

✅ GPU 활성화됨: Tesla T4
Thu Oct 23 07:19:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


✅ 모델이 GPU에 로드됨: cuda:0
customdata.txt 파일이 생성되었습니다.
Max length: 32
Trainable parameters: 2,086,979,328 (100.00% of total)
⚡ Mixed Precision Training (FP16) 활성화

🎯 학습 시작!
📊 설정: Batch size=8, Accumulation steps=2
📊 효과적인 배치 사이즈: 16
📊 Learning rate=3e-05, Epochs=5
📊 총 학습 스텝: 10
--------------------------------------------------


/tmp/ipython-input-3900161364.py:175: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-3900161364.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/5] Step [0/2] Loss: 6.2095 LR: 3.00e-05


ValueError: Attempting to unscale FP16 gradients.